In [1]:
# Import required libraries

import pandas as pd
import numpy as np
import math as mt
from datetime import date, datetime, timedelta
from IPython.display import clear_output
import scipy.stats as ss
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, PowerTransformer, OneHotEncoder

In [2]:
# Get cwd as this changes depending on which laptop is being used.
# Could probably use a relative reference but this may restrict development.
# Import all required data from CSV files.

directory = os.getcwd()

growth_data_path = f"{directory}\\growth_db.csv"
weather_data_path = f"{directory}\\weather_db.csv"
zone_data_path = f"{directory}\\zone_db.csv"
    
growth_data = pd.read_csv(growth_data_path)
weather_data = pd.read_csv(weather_data_path)
zone_data = pd.read_csv(zone_data_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Tom\\Documents\\GitHub\\leek-growth-model\\zone_db.csv'

In [ ]:
# Ensure all data within a df is visible when printed.

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Variables used within script.

linearisation_power = 0.625 # This is used to transform mean_diameter so it has a linear relationship with solar/heat.
stripping_coef = 0.92 # This is to allow for a slight reduction in diameter once leeks are stripped at harvest.
min_grow_temp = 3 # The minimum temperature that leeks will grow at. Needed to calculate heat units.
max_grow_temp = 27 # The temperature at which maximum growth rate is achieved. Needed to calculate heat units.
season = datetime(2022, 1, 1) # Output data only required for fields with a planting date in this season.

In [ ]:
# Feature creation, alteration and exploration for weather data.

weather_data['date'] = pd.to_datetime(weather_data['date'], format='%d/%m/%Y')
weather_data['time'] = pd.to_datetime(weather_data['time'], format='%H:%M:%S')
weather_data['day'] = weather_data.date.dt.day
weather_data['month'] = weather_data.date.dt.month
weather_data['day_month'] = weather_data['day'].astype(str) + " - " + weather_data['month'].astype(str) # This is used to calculate weather averages.

# Calculate heat_units using min and max temperature variables.

weather_data['heat_units'] = weather_data['avg_temp'] - min_grow_temp
weather_data['heat_units'] = np.where((weather_data['heat_units'] < 0), 0, weather_data['heat_units'])
weather_data['heat_units'] = np.where((weather_data['heat_units'] > max_grow_temp - min_grow_temp), 1, weather_data['heat_units']/24)

weather_data.info()

In [ ]:
# Investigate if any dates have something other than 24 datapoints as this must be an error.

weather_data.date.value_counts()[weather_data.date.value_counts()!=24]

In [ ]:
weather_data.head()

In [ ]:
# Feature creation, alteration and exploration for growth data.

growth_data['sample_date'] = pd.to_datetime(growth_data['sample_date'], format='%d/%m/%Y')
growth_data['fieldzone'] = growth_data["field"] + " - " + growth_data["zone"].astype(str)
growth_data['stripped_diameter'] = growth_data['diameter'] * stripping_coef

growth_data.info()

In [ ]:
growth_data.diameter.describe()

In [ ]:
growth_data.head()

In [ ]:
# Feature creation, alteration and exploration for zone data.

zone_data['planting_date'] = pd.to_datetime(zone_data['planting_date'], format='%d/%m/%Y')
zone_data['harvest_date'] = pd.to_datetime(zone_data['harvest_date'], format='%d/%m/%Y')
zone_data['zone'] = zone_data['zone'].astype(int)
zone_data["fieldzone"] = zone_data["field"] + " - " + zone_data["zone"].astype(str)
zone_data["fieldvariety"] = zone_data["field"] + " - " + zone_data["variety"]

zone_data.info()

In [ ]:
zone_data.describe()

In [ ]:
zone_data.head()

In [ ]:
# Start to build summary dataframe
# This dataframe will be aggregated to fit models. Need to aggregate explained later.

summary_data = growth_data.copy()

summary_data["zone"] = summary_data["zone"].astype(str)
summary_data["fieldzone"] = summary_data["field"] + " - " + summary_data["zone"]
summary_data['fieldzonedate'] = summary_data['fieldzone'] + " - " + summary_data['sample_date'].astype(str)

summary_data = summary_data.set_index('fieldzone')
summary_data = summary_data.join(zone_data.set_index('fieldzone'), rsuffix = '_join')

summary_data['fieldvarietydate'] = summary_data['fieldvariety'] + " - " + summary_data['sample_date'].astype(str)
summary_data['heat_units'] = 0
summary_data['solar_radiation'] = 0

summary_data.info()

In [ ]:
# Reset index and remove join columns from summary dataframe.

summary_data = summary_data.reset_index(inplace=False)
summary_data = summary_data.drop(columns=['field_join', 'zone_join'], inplace=False)
summary_data.head()

In [ ]:
# Skewness is how far the mode is to the right (positive) or left (negative) of the mean.
# Kurtosis is the length of the tails on the distribution. Normal distribution has kurtosis of 3.

def skewness(series):
    """Aggregate function to return skew of distribution"""
    return ss.skew(series, bias = False)

def kurt(series):
    """Aggregate function to return kurtosis of distribution"""
    return ss.kurtosis(series, bias = False)

In [ ]:
# Aggregation of summary data to which can be used to fit models.
# Non aggregated data can also be used to fit models but it will be more difficult to create a predicted distribution...
# cont... due to not having a standard deviation variable which is generated during aggregation.

summary_data_avg = summary_data.copy()

summary_data_avg = summary_data_avg.groupby(['fieldzonedate']).agg({'stripped_diameter' : ['mean', 'std', 'count', skewness, kurt],
                                                                    'method' : ['first'],
                                                                    'inputs' : ['first'],
                                                                    'variety' : ['first'],
                                                                    'protection' : ['first'],
                                                                    'sand' : ['mean'],
                                                                    'silt' : ['mean'],
                                                                    'clay' : ['mean'],
                                                                    'organic_matter' : ['mean'],
                                                                    'planting_rate' : ['first'],
                                                                    'planting_date' : ['first'],
                                                                    'sample_date' : ['first'],
                                                                    'fieldzone' : ['first']}).reset_index()

summary_data_avg.columns = ['fieldzonedate',
                            'mean_diameter',
                            'std_dev_diameter',
                            'pp2m2',
                            'skewness',
                            'kurtosis',
                            'method',
                            'inputs',
                            'variety',
                            'protection',
                            'sand',
                            'silt',
                            'clay',
                            'organic_matter',
                            'planting_rate',
                            'planting_date',
                            'sample_date',
                            'fieldzone']

summary_data_avg['field'] = summary_data_avg['fieldzone'].str.split(' - ').str[0]
summary_data_avg['zone'] = summary_data_avg['fieldzone'].str.split(' - ').str[1]
summary_data_avg['d_lin'] = (summary_data_avg['mean_diameter'])**linearisation_power
summary_data_avg['s_lin'] = (summary_data_avg['std_dev_diameter'])**linearisation_power
summary_data_avg['heat_units'] = 0
summary_data_avg['solar_radiation'] = 0

summary_data_avg.tail()

In [ ]:
summary_data_avg[['method', 'inputs', 'variety', 'protection', 'fieldzone', 'field']].describe()

In [ ]:
summary_data_avg[['planting_date', 'sample_date']].describe(datetime_is_numeric=True)

In [ ]:
summary_data_avg[['mean_diameter', 'std_dev_diameter', 'pp2m2', 'skewness', 'kurtosis', 'sand', 'silt', 'clay', 'organic_matter', 'planting_rate']].describe()

In [ ]:
# Aggregation of weather data to create a more manageable dataframe as we only need day by day accuracy not hour by hour.

weather_data_avg = weather_data.copy()

weather_data_avg = weather_data_avg.groupby(['date']).agg({'rain' : ['sum'],
                                                       'heat_units' : ['sum'],
                                                       'solar_radiation' : ['sum'],
                                                       'wind_speed_avg' : ['mean'],
                                                       'rh' : ['mean'],
                                                       'avg_temp' : ['mean']}).reset_index()

weather_data_avg.columns = ['date',
                            'rain',
                            'heat_units',
                            'solar_radiation',
                            'wind_speed_avg',
                            'rh',
                            'avg_temp']

weather_data_avg['day'] = weather_data_avg.date.dt.day
weather_data_avg['month'] = weather_data_avg.date.dt.month
weather_data_avg['day_month'] = weather_data_avg['day'].astype(str) + " - " + weather_data_avg['month'].astype(str)

# Further aggregation of weather to get the average weather for a given day and a given month, regardless of the year.
# This average weather will be used to predict heat and solar that a plant will receive.
# Could possibly introduce some sort of short term weather forecast here???

weather_data_avg_group = weather_data_avg.copy()

weather_data_avg_group = weather_data_avg_group.groupby(['day_month']).agg({'rain' : ['mean'],
                                                                            'heat_units' : ['mean'],
                                                                            'solar_radiation' : ['mean'],
                                                                            'wind_speed_avg' : ['mean'],
                                                                            'rh' : ['mean'],
                                                                            'avg_temp' : ['mean']}).reset_index()

weather_data_avg_group.columns = ['day_month',
                                  'rain',
                                  'heat_units',
                                  'solar_radiation',
                                  'wind_speed_avg',
                                  'rh',
                                  'avg_temp']

# Extend aggregated weather data by adding 300 days of predicted weather.

max_date = max(weather_data_avg.date)

for i in range(1, 300):
    
    clear_output(wait=True)
    
    date = max_date + timedelta(days=i)
    weather_data_avg = weather_data_avg.append({'date': date,
                                                'rain': np.nan,
                                                'heat_units':np.nan,
                                                'solar_radiation':np.nan,
                                                'wind_speed_avg':np.nan,
                                                'rh':np.nan,
                                                'avg_temp':np.nan }, ignore_index=True)
    
    print("Current Progress:", np.round(i/300*100,0),"%")
    
weather_data_avg['day'] = weather_data_avg.date.dt.day
weather_data_avg['month'] = weather_data_avg.date.dt.month
weather_data_avg['day_month'] = weather_data_avg['day'].astype(str) + " - " + weather_data_avg['month'].astype(str)  

weather_data_avg.tail()

In [ ]:
weather_data_avg[weather_data_avg['solar_radiation']//1000000 != 0].head()

In [ ]:
###THIS IS A VERY SLOW PROCESS (COULD PARALLEL PROCESSING BE INTRODUCED???)

def mean_weather(day_month, variable):
    """function that calculates the average value for a weather variable for a given day within a given month"""
    df = weather_data_avg_group[weather_data_avg_group['day_month']==day_month]
    weather_value = df[variable].sum()
    return weather_value

for variable in ['rain', 'heat_units', 'solar_radiation', 'wind_speed_avg', 'rh', 'avg_temp']:
    for i in weather_data_avg.index:
        
        clear_output(wait=True)
        
        # If statement so only future dates have weather predicted.
        # Tried == np.nan, but that didn't work so workaround implemented. Make sure no variable has a value larger than 'MEGA_NUM'.
        
        MEGA_NUM = 10000000
        
        if weather_data_avg[variable][i]//MEGA_NUM != 0:
            day_month = weather_data_avg['day_month'][i]
            weather_data_avg[variable][i] = mean_weather(day_month, variable)
            
        print(f"{variable} progress:", np.round(i/len(weather_data_avg)*100,0),"%")
    
weather_data_avg.tail()

In [ ]:
weather_data_avg.info()

In [ ]:
def cumulative_weather(start, finish, weather_variable, weather_data):
    """Function used to find the cumulative weather input between 2 given dates"""
    df = weather_data.loc[(weather_data['date'] > start) & (weather_data['date'] < finish), [weather_variable]]
    total_units = df[weather_variable].sum()
    return total_units

In [ ]:
# Overwrite solar radiation with the cumulative total that has been received between planting and sampling.

for i in summary_data_avg.index:
    
    clear_output(wait=True)
    
    planting_date = summary_data_avg['planting_date'][i]
    sample_date = summary_data_avg['sample_date'][i]
    summary_data_avg['solar_radiation'][i] = cumulative_weather(planting_date, sample_date, 'solar_radiation', weather_data_avg)
    
    print("Current Progress:", np.round(i/len(summary_data_avg)*100,0),"%")

In [ ]:
# Overwrite heat units with the cumulative total that has been received between planting and sampling.

for i in summary_data_avg.index:
    clear_output(wait=True)
    
    planting_date = summary_data_avg['planting_date'][i]
    sample_date = summary_data_avg['sample_date'][i]
    summary_data_avg['heat_units'][i] = cumulative_weather(planting_date, sample_date, 'heat_units', weather_data_avg)
    
    print("Current Progress:", np.round(i/len(summary_data_avg)*100,0),"%")

In [ ]:
# Dummy code for protection column while retaining the original 'protection' series within the dataframe for visualisation purposes.
# This allows a larger amount of data to be used within Lin Reg model as df does not need to be filtered based on protection.

summary_data_avg['protection_2'] = summary_data_avg['protection'].copy()
summary_data_avg = pd.get_dummies(summary_data_avg, columns = ['protection'], drop_first = False)
summary_data_avg.rename(columns={'protection_2': 'protection'}, inplace=True)

In [ ]:
summary_data_avg.info()

In [ ]:
# Box plots to check for outliers in continuous variables of importance.

sns.boxplot(x = 'inputs', y = 'pp2m2', data = summary_data_avg, hue = 'method', orient = 'v')
plt.legend(bbox_to_anchor=(0.45, 0.97), loc='upper left', borderaxespad=0)
plt.show()

sns.boxplot(x = 'method', y = 'heat_units', data = summary_data_avg, orient = 'v')
plt.show()

sns.boxplot(x = 'method', y = 'solar_radiation', data = summary_data_avg, orient = 'v')
plt.show()

sns.boxplot(x = 'method', y = 'mean_diameter', data = summary_data_avg, orient = 'v')
plt.show()

sns.boxplot(x = 'method', y = 'std_dev_diameter', data = summary_data_avg, orient = 'v')
plt.show()

sns.boxplot(data = summary_data_avg[['sand', 'silt', 'clay', 'organic_matter']])
plt.show()

In [ ]:
# Value counts of categorical variables of importance.

summary_data_avg['variety'].value_counts().plot(kind = 'bar', xlabel = 'variety', ylabel = 'count')
plt.figure(figsize = (10, 8))
plt.show()

summary_data_avg['method'].value_counts().plot(kind = 'bar', xlabel = 'method', ylabel = 'count')
plt.figure(figsize = (10, 8))
plt.show()

summary_data_avg['inputs'].value_counts().plot(kind = 'bar', xlabel = 'inputs', ylabel = 'count')
plt.figure(figsize = (10, 8))
plt.show()

summary_data_avg['protection'].value_counts().plot(kind = 'bar', xlabel = 'protection', ylabel = 'count')
plt.figure(figsize = (10, 8))
plt.show()


In [ ]:
# Check for any null values and if any are present, find out why.

summary_data_avg[summary_data_avg.isna().any(axis=1)]

In [ ]:
#Remove any null values to prevent failure but if any are present, it's essential to find out why.

summary_data_avg = summary_data_avg.dropna()

In [ ]:
def average_count(fieldzone, df_1 = summary_data_avg):
    """Function used to find the average plants per two meters squared from every sample from a given fieldzone over the entire season"""
    
    df_1 = df_1[df_1['fieldzone']==fieldzone]
    average_count = df_1['pp2m2'].mean()
    
    if mt.isnan(average_count):
        average_count = 40
      
    return average_count

int(average_count('RH33 - 1')) == 36

In [ ]:
def max_sample_date(fieldzone, df_1 = summary_data_avg, df_2 = zone_data):
    """Function used to find the most recent sample date for a given fieldzone"""
    
    df_1 = df_1[df_1['fieldzone'] == fieldzone]
    max_sample_date = max(df_1['sample_date'], default = 0)
    if max_sample_date == 0:
        df_2 = df_2[df_2['fieldzone'] == fieldzone]
        max_sample_date = df_2['planting_date'].max()
    
    return max_sample_date

max_sample_date('RH33 - 1')

In [ ]:
def max_mean_diameter_lin(fieldzone, df_1 = summary_data_avg):
    """Function used to find the mean diameter of the sample at the most recent sample date"""
    
    df_1 = df_1[df_1['fieldzone']==fieldzone]
    max_mean_diameter = df_1['mean_diameter'].max()
    max_mean_diameter_lin = max_mean_diameter ** linearisation_power
    
    if mt.isnan(max_mean_diameter_lin):
        max_mean_diameter_lin = 0
    
    return max_mean_diameter_lin

int(max_mean_diameter_lin('Allans 07 - 1')) == 9

In [ ]:
def max_std_dev_diameter_lin(fieldzone, df_1 = summary_data_avg):
    """Function used to find the standard deviation of the sample at the most recent sample date"""
    
    df_1 = df_1[df_1['fieldzone']==fieldzone]
    max_std_dev_diameter = df_1['std_dev_diameter'].max()
    max_std_dev_diameter_lin = max_std_dev_diameter ** linearisation_power
    
    if mt.isnan(max_std_dev_diameter_lin):
        max_std_dev_diameter_lin = 0
    
    return max_std_dev_diameter_lin

int(max_std_dev_diameter_lin('Allans 07 - 1')) == 4

In [ ]:
def max_solar(fieldzone, df_1 = summary_data_avg):
    """Function used to find the solar radiation received at the most recent sample date"""
    
    df_1 = df_1[df_1['fieldzone']==fieldzone]
    max_solar = df_1['solar_radiation'].max()
    
    if mt.isnan(max_solar):
        max_solar = 0
    
    return max_solar

max_solar('Allans 07 - 1') == 1600496

In [ ]:
def max_heat(fieldzone, df_1 = summary_data_avg):
    """Function used to find the heat units received at the most recent sample date"""
    
    df_1 = df_1[df_1['fieldzone']==fieldzone]
    max_heat = df_1['heat_units'].max()
    
    if mt.isnan(max_heat):
        max_heat = 0
    
    return max_heat

max_heat('Allans 07 - 1') == 2464

In [ ]:
def filter_data(data, method, inputs, variety):
    """Function used to filter df so it only contains a single variety, input & method"""
    
    filtered = data[data['variety'].str.contains(variety)]
    filtered = filtered[filtered['inputs'].str.contains(inputs)]
    filtered = filtered[filtered['method'].str.contains(method)]
    
    return filtered

In [ ]:
def predict_weather(start, finish, variable, df_1 = weather_data):
    """Function used to calculated a predicted weather variable for a given timeframe"""
    
    df_1 = df_1.loc[(df_1['date'] > start) & (df_1['date'] < finish), [variable]]
    predicted_weather= df_1[variable].sum()
    
    return predicted_weather

start = datetime(year=2021, month=6, day=2, hour=13, minute=14, second=31)
finish = datetime(year=2022, month=6, day=2, hour=13, minute=14, second=31)

predict_weather(start, finish, 'rain') == 434

In [ ]:
# This df will be used for Linear Regression model and Visualisation. Prediction will be taken from the most recent (maximum) sample date...
# The 'max' variables just indicate the result of the most recent sample at that zone.

zone_data['mean_pp2m2'] = 0.0
zone_data['max_sample_date'] = 0
zone_data['max_mean_diameter_lin'] = 0.0
zone_data['max_std_dev_diameter_lin'] = 0.0
zone_data['max_heat'] = 0.0
zone_data['max_solar'] = 0.0
zone_data['remaining_heat'] = 0.0
zone_data['remaining_solar'] = 0.0
zone_data['rain_after_planting'] = 0

for i in zone_data.index:
    
    clear_output(wait=True)
    
    fieldzone = zone_data.loc[i, 'fieldzone']
    zone_data.loc[i, 'mean_pp2m2'] = average_count(fieldzone)
    zone_data.loc[i, 'max_sample_date'] = max_sample_date(fieldzone)
    zone_data.loc[i, 'max_mean_diameter_lin'] = max_mean_diameter_lin(fieldzone)
    zone_data.loc[i, 'max_std_dev_diameter_lin'] = max_std_dev_diameter_lin(fieldzone)
    zone_data.loc[i, 'max_heat'] = max_heat(fieldzone)
    zone_data.loc[i, 'max_solar'] = max_solar(fieldzone)
    start = zone_data.loc[i, 'max_sample_date']
    finish = zone_data.loc[i, 'harvest_date']
    rain_start = zone_data.loc[i, 'planting_date']
    rain_finish = rain_start + timedelta(days = 14)
    zone_data.loc[i, 'remaining_heat'] = cumulative_weather(start, finish, 'heat_units', weather_data_avg)
    zone_data.loc[i, 'remaining_solar'] = cumulative_weather(start, finish, 'solar_radiation', weather_data_avg)
    zone_data.loc[i, 'rain_after_planting'] = cumulative_weather(rain_start, rain_finish, 'rain', weather_data_avg)  
    
    print("Current Progress:", np.round(i/len(zone_data)*100,0),"%")
    
zone_data['establishment'] = (zone_data['mean_pp2m2']/2*10000)/zone_data['planting_rate']

In [ ]:
# These columns will be populated within the Lin Reg script.

zone_data['est_mean_diameter_gain'] = 0.0
zone_data['est_std_dev_diameter_gain'] = 0.0
zone_data['est_mean_diameter'] = 0.0
zone_data['est_std_dev_diameter'] = 0.0

In [ ]:
#Custom Transformer

class CustomTransformer(BaseEstimator, TransformerMixin):
    #Class Constructor 
    def __init__(self, method):
        self.cat_column_list = list(X.select_dtypes(include=["object_"]))
        self.method = method
       
    def fit(self, X, y = None):

        if self.method.lower() = 'ordinal':
            self.trns = OrdinalEncoder().fit(X[self.column_list])
            
        if self.method.lower() = 'onehot':
            self.trns = OneHotEncoder().fit(X[self.column_list])

        return self
        

    def transform(self, X, y = None):
        
        pd.options.mode.chained_assignment = None  # default='warn'
        trns_array = self.trns.transform(X[self.column_list])
        X.loc[:, self.column_list] = trns_array.copy()

        return X
        
        
    def inverse_transform(self, X, y = None):
        
        pd.options.mode.chained_assignment = None  # default='warn'
        trns_array = self.trns.inverse_transform(X[self.column_list])
        X.loc[:, self.column_list] = trns_array.copy()
            
        return X